In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import sys
sys.path.append('../definitions')
import def_model as mod_def
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

Select Dates

In [2]:
start_date = pd.Timestamp('2024-03-20')
end_date = pd.Timestamp('2024-07-09')
n_lags = 5

Import data

In [3]:
df = mod_def.combine_resampled_data(start_date, end_date)

print(df.head())
print(f"This is the shape:" ,df.shape)

Proportion of storm to no storm is:  0.12 %
    NS_SQUID   Z_SQUID   NS_Fluxgate  EW_Fluxgate    Z_Fluxgate   H Component  \
0 -17.218172  0.263233  10934.303393   -42.359410 -22656.440907  10934.385443   
1 -17.199095  0.317575  10934.335773   -42.639808 -22656.444363  10934.418915   
2 -17.069690  0.255518  10934.388935   -42.276737 -22656.497270  10934.470664   
3 -17.053298  0.289818  10934.401460   -42.308557 -22656.484653  10934.483313   
4 -17.142593  0.255207  10934.392687   -42.265653 -22656.573902  10934.474373   

   flag  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  
This is the shape: (817967, 7)


Use test data

In [4]:
# Example multivariate time series data (e.g., 3 variables)
np.random.seed(42)
n_samples = 100
n_features = 3
data = np.random.rand(n_samples, n_features)

In [14]:
print(df.iloc[:, 2:7])
X, Y = mod_def.create_lagged_features(df.iloc[:, 2:7], n_lags=n_lags)
print(f"This is the head of X", X)
# print(f"This is the head of Y", Y)

# Train-test split (without shuffling to maintain temporal order)
test_size = 0.2
split_idx = int((1 - test_size) * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
Y_train, Y_test = Y[:split_idx], Y[split_idx:]

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

       NS_Fluxgate  EW_Fluxgate    Z_Fluxgate   H Component  flag
0     10934.303393   -42.359410 -22656.440907  10934.385443   0.0
1     10934.335773   -42.639808 -22656.444363  10934.418915   0.0
2     10934.388935   -42.276737 -22656.497270  10934.470664   0.0
3     10934.401460   -42.308557 -22656.484653  10934.483313   0.0
4     10934.392687   -42.265653 -22656.573902  10934.474373   0.0
...            ...          ...           ...           ...   ...
7339  10961.682807   -86.046363 -22642.759082  10962.020523   0.0
7340  10961.719243   -85.969953 -22642.758770  10962.056358   0.0
7341  10961.747277   -86.003307 -22642.731825  10962.084653   0.0
7342  10961.737595   -85.912318 -22642.798250  10962.074258   0.0
7343  10961.745295   -85.905466 -22642.819408  10962.081903   0.0

[817967 rows x 5 columns]
This is the head of X [[ 10934.39268667    -42.26565333 -22656.57390167 ...    -42.23941
  -22656.46315333  10934.46896531]
 [ 10934.38738       -42.23941    -22656.46315333 ...    

In [15]:
kernel_list = ['linear', 'rbf', 'poly']

for kernel_i in kernel_list:
    # Initialize the SVC model (Support Vector Classification)
    svc = SVC(kernel=kernel_i, class_weight='balanced')

    # Train the SVC model
    svc.fit(X_train_scaled, Y_train)

    # Make predictions
    Y_pred = svc.predict(X_test_scaled)

    # Evaluate the model using classification metrics
    accuracy = accuracy_score(Y_test, Y_pred)
    print(f"Accuracy for model {kernel_i}: {accuracy:.4f}")

    # Display detailed classification metrics
    print(f"Classification Report for model {kernel_i}:")
    print(classification_report(Y_test, Y_pred))

    plt.plot(Y_test, label="True")
    plt.plot(Y_pred, label="Predicted", linestyle='--')
    plt.title(f"True vs Predicted Binary Labels for model {kernel_i}")
    plt.legend()
    plt.show()

Create training and testing data

In [ ]:
train_end = len(df)*0.7
Y = pd.DataFrame()
X = pd.DataFrame()
Y = df['flag'] # select labels 
X = df.drop('flag', axis=1) #select features
X_train = X.loc[0:train_end]
X_test = X.loc[train_end:]
Y_train = Y.loc[0:train_end]
Y_test = Y.loc[train_end:]

print(f"The data is split at 70%")
print("This is X train head:\n",X_train.head())
print("This is X train shape:\n",X_train.shape)
print("This is y test head:\n",Y_test.head())
print("This is y test shape:\n",Y_test.shape)
print(type(X_train), type(Y_test))

Scale

In [ ]:
scaler_X = StandardScaler()
scaler_Y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
Y_train_scaled = scaler_Y.fit_transform(Y_train)
Y_test_scaled = scaler_Y.transform(Y_test)

Initialize the base SVR model

In [ ]:
svr = SVR(kernel='rbf')
multi_output_svr = MultiOutputRegressor(svr)